# Normalization and units

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import astropy.units as u
from astropy.visualization import quantity_support

from celerite2 import terms, GaussianProcess

from gadfly import PowerSpectrum
from gadfly.psd import ppm, to_psd_units

# Set example hyperparameters for testing:
S0 = 1 * ppm**2 / u.uHz
f0 = 3090 * u.uHz
w0 = 2 * np.pi * f0
Q = 100

# Define a SHO kernel:
kernel = terms.SHOTerm(
    S0=S0.to(ppm**2 / u.uHz).value, 
    w0=w0.to(u.uHz).value, 
    Q=Q
)

# Define a GP with this kernel:
gp = GaussianProcess(kernel)

# Times are generated here in [days]:
t = np.linspace(0, 10, int(5e4)) * u.d

# Since GP hyperparameters are in [uHz], convert the time
# axis into the inverse of that unit, [uHz^-1]:
t_celerite2 = t.to(1/u.uHz).value

# Now compute the GP with the times in the correct units:
gp.compute(t_celerite2, check_sorted=False)

# Signal generated by celerite2 has [ppm]:
y = gp.sample() * ppm

In [ ]:
def sho_psd(freq, S0, w0, Q):
    """
    Stochastically driven, dampled harmonic oscillator.
    
    The normalization of this PSD equation differs from the
    description in the celerite2 docs by a factor of :math:`2\pi`.
    
    """
    # convert physical to angular frequency
    w = 2 * np.pi * freq
        
    # What follows is the usual celerite2 SHO PSD:
    return np.sqrt(2/np.pi) * S0 * w0**4 / ((w**2 - w0**2)**2 + (w**2 * w0**2 / Q**2))

In [ ]:
# Measure the observed power spectrum via FFT:
d = t[1] - t[0]
freq = np.fft.rfftfreq(len(y), d)
fft = np.fft.rfft(y.to(ppm).value)

# The FFT must be normalized by this factor:
fft_normalization = d / ((2*np.pi) ** 0.5 * len(y))

# The power spectrum in the usual asteroseismic units:
power_fft = to_psd_units(np.real(fft * np.conj(fft)) * ppm**2 * fft_normalization)

# Compute the power spectrum directly from 
# celerite2's SHOTerm.get_psd method:
power_celerite2 = kernel.get_psd(
    2 * np.pi * freq.to(u.uHz).value
)

In [ ]:
# bin the simulated power spectrum for comparison with the 
# input kernel for checking the round-tripping. Skip ``freq==0``.
binned_fft = PowerSpectrum(
    freq.to(u.uHz)[1:], 
    to_psd_units(power_fft)[1:], 
    name='test'
).bin(100)

# Plot them all and show they align:
with quantity_support():
    plt.loglog(freq.to(u.uHz), power_fft, '.', label='Power spectrum', ms=3, alpha=0.1, color='silver');
    binned_fft.plot(ax=plt.gca(), p_mode_inset=False, label_obs='Binned power spectrum')
    plt.loglog(freq.to(u.uHz), to_psd_units(power_celerite2), marker=',', label='SHO (from celerite2)')
    plt.loglog(freq.to(u.uHz), sho_psd(freq.to(u.uHz), S0, w0, Q), ':', label='SHO (manual)')
    plt.axvline(w0/(2*np.pi))
plt.legend()
plt.ylim([1e-4, 1e4]);